### MLP多层感知机

In [5]:
from utils.dataset_utils import get_classes_indexes_counts
from sklearn.metrics import confusion_matrix
# 导入必要的库
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
import scipy.io as sio  # 从.mat文件中读取数据集
# 加载鸢尾花数据集
# data = load_iris()
mat_data = sio.loadmat('../data/dataset/Satellite.mat')
X = mat_data['X']  # 特征
y = mat_data['Y']  # 标签

# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)
classes, counts = get_classes_indexes_counts(y_test)
print(counts)
# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 构建并训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(10, 20), max_iter=1000, random_state=100)
mlp.fit(X_train, y_train)
index_pred_proba=mlp.predict_proba(X_test)
# 预测和评估模型
y_pred = mlp.predict(X_test)

# 输出结果
print("准确率:", accuracy_score(y_test, y_pred))
print("\n分类报告:\n", classification_report(y_test, y_pred))
# 打印混淆矩阵
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print(index_pred_proba.shape)
index_pred_proba

[423 227 423 187 228 443]


D:\IDE\Anaconda\envs\pytorch\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


准确率: 0.893837389953392

分类报告:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       423
           1       0.96      0.98      0.97       227
           2       0.90      0.92      0.91       423
           3       0.70      0.62      0.66       187
           4       0.88      0.90      0.89       228
           5       0.86      0.86      0.86       443

    accuracy                           0.89      1931
   macro avg       0.88      0.88      0.88      1931
weighted avg       0.89      0.89      0.89      1931

Confusion Matrix:
[[412   0   7   0   4   0]
 [  0 222   1   0   4   0]
 [  5   0 388  16   0  14]
 [  2   2  27 116   1  39]
 [  5   5   0   5 206   7]
 [  1   3   8  29  20 382]]
(1931, 6)


array([[5.17323587e-17, 1.60220408e-13, 5.19973066e-08, 3.94168485e-12,
        9.99953140e-01, 4.68078246e-05],
       [2.61268635e-10, 2.07901013e-13, 9.00869586e-13, 9.67447152e-01,
        3.06865848e-06, 3.25497789e-02],
       [6.37113520e-08, 5.19648702e-07, 8.65279416e-05, 4.22702150e-01,
        7.46518694e-04, 5.76464220e-01],
       ...,
       [9.99991971e-01, 1.50201163e-10, 8.12689687e-10, 2.13656185e-15,
        8.02837161e-06, 1.07145910e-12],
       [2.08331479e-08, 3.35203027e-07, 1.52264669e-02, 8.54921174e-01,
        2.10410717e-05, 1.29830962e-01],
       [1.21878128e-07, 1.53394928e-09, 1.00749707e-01, 2.51769002e-01,
        8.81122826e-07, 6.47480286e-01]])

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# 加载鸢尾花数据集
iris = load_iris()
X, y = iris.data, iris.target

# 数据集分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_test.shape)
print(y_test.shape)
# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 创建和训练MLP模型
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# 预测
y_pred = mlp.predict(X_test)

# 输出结果
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=iris.target_names)



(45, 4)
(45,)
